#### Imports

In [15]:
import os
from pathlib import Path
import copy
from typing import TypeAlias
import numpy as np
import numpy.typing as npt
import py4spice as spi
from py4spice import globals_types

#### Constants and directories

In [16]:
# Path to NGSPICE
NGSPICE_EXE = Path("/usr/bin/ngspice")

# Path to my project
PROJ_PATH = Path("/workspaces/sw_pwr_03/circuits/simple_rc")

# where to put results
RESULTS_PATH = PROJ_PATH / "results"

# netlist file names
NETLISTS_PATH = PROJ_PATH / "netlists"
TOP_FILENAME = NETLISTS_PATH / "top.cir"
DUT_FILENAME = NETLISTS_PATH / "dut.cir"
LOAD_FILENAME = NETLISTS_PATH / "load.cir"
STIMULUS_FILENAME = NETLISTS_PATH / "stimulus.cir"
SUPPLIES_FILENAME = NETLISTS_PATH / "supplies.cir"
CONTROL_FILENAME = NETLISTS_PATH / "control.cir"

#### Create netlists objects. We'll combine them after "control" netlist is constructed.

In [17]:
dut = spi.Netlist(NETLISTS_PATH / DUT_FILENAME)
load = spi.Netlist(NETLISTS_PATH / LOAD_FILENAME)
supplies = spi.Netlist(NETLISTS_PATH / SUPPLIES_FILENAME)
stimulus = spi.Netlist(NETLISTS_PATH / STIMULUS_FILENAME)

#### Create signal vectors

In [18]:
VEC_ALL = spi.Vectors("all")
VEC_INTEREST = spi.Vectors("in mid out")

#### Define analyses

In [19]:
list_of_analyses: list[spi.Analyses] = []

# 1st analysis: operating point
op_cmd = "op"
op1 = spi.Analyses("op1", "op", op_cmd, VEC_ALL, RESULTS_PATH)
list_of_analyses.append(op1)

# 2nd analysis: transient with time-end = 20m
tr_cmd = "tran 1e-3 20e-3"
tr1 = spi.Analyses("tr1", "tran", tr_cmd, VEC_ALL, RESULTS_PATH)
list_of_analyses.append(tr1)

#### Create control section

In [20]:
my_control = spi.Control()  # create 'my_control' object
my_control.insert_lines(["listing"])  # cmd to list out netlist
for analysis in list_of_analyses:  # statements for all analyses
    my_control.insert_lines(analysis.lines_for_cntl())
spi.print_section("Control File", my_control)  # print out contents
my_control.content_to_file(CONTROL_FILENAME)  # creat the actual file
control = spi.Netlist(NETLISTS_PATH / CONTROL_FILENAME)


--- Control File ---
.control
* Timestamp: Fri Feb  2 04:18:09 2024
set wr_singlescale  $ makes one x-axis for wrdata
set wr_vecnames     $ puts names at top of columns
listing
op
print line all > /workspaces/sw_pwr_03/circuits/simple_rc/results/op1.txt
tran 1e-3 20e-3
wrdata /workspaces/sw_pwr_03/circuits/simple_rc/results/tr1.txt all
quit
.endc
--------------------



#### Simulate

In [21]:
# Combine the netlists and write out into one top netlist, ready to simulate
top = dut + load + supplies + stimulus + control
top.writefile(TOP_FILENAME)
spi.print_section("top netlist", top)

sim1 = spi.Simulate(NGSPICE_EXE, TOP_FILENAME)
spi.print_section("Ngspice Command", sim1)
sim1.run()


--- top netlist ---
R1 IN MID 1K
C1 COM MID 0.1U
R2 MID OUT 100
RLOAD OUT COM 100K
VCOM COM 0 0
* VIN IN 0 1
* PULSE (V1 V2 TD TR TF PW PER NP)
VIN IN 0 PULSE (0 10 0 3N 3N 100M)
.op
.control
* Timestamp: Fri Feb 2 04:18:09 2024
set wr_singlescale $ makes one x-axis for wrdata
set wr_vecnames $ puts names at top of columns
listing
op
print line all > /workspaces/sw_pwr_03/circuits/simple_rc/results/op1.txt
tran 1e-3 20e-3
wrdata /workspaces/sw_pwr_03/circuits/simple_rc/results/tr1.txt all
quit
.endc
-------------------


--- Ngspice Command ---
/usr/bin/ngspice -b /workspaces/sw_pwr_03/circuits/simple_rc/netlists/top.cir
-----------------------


Note: No compatibility mode selected!


Circuit: r1 in mid 1k

	r1 in mid 1k

     1 : r1 in mid 1k
     2 : .global gnd
     3 : c1 com mid 0.1u
     4 : r2 mid out 100
     5 : rload out com 100k
     6 : vcom com 0 0
     9 : vin in 0 pulse (0 10 0 3n 3n 100m)
    10 : .op
    11 : .control
    13 : set wr_singlescale
    14 : set wr_vecna

#### Simulation results to numpy

In [23]:
sim_results = [
    spi.SimResults.from_file(analysis.cmd_type, analysis.results_filename)
    for analysis in list_of_analyses
]

dup indexes are: [5]


#### First simulation results: operating point

In [26]:
# 1st simulation was operating point
spi.print_section("Operating Point Results", sim_results[0].print_table())


0.0

--- Operating Point Results ---
com           0.0
in            0.0
mid           0.0
out           0.0
vcom#branch   0.0
vin#branch    0.0
-------------------------------



#### Second simulation results: transient

In [ ]:
# 2nd simulation was transient
plot_tr(tr1, VEC_ALL, RESULTS_PATH)